In [1]:
!git clone https://github.com/openai/shap-e

Cloning into 'shap-e'...
remote: Enumerating objects: 336, done.
remote: Counting objects: 100% (260/260), done.
remote: Compressing objects: 100% (241/241), done.
remote: Total 336 (delta 41), reused 218 (delta 18), pack-reused 76
Receiving objects: 100% (336/336), 11.72 MiB | 15.57 MiB/s, done.
Resolving deltas: 100% (44/44), done.


In [2]:
%cd shap-e

/content/shap-e


In [3]:
pip install -e .

Obtaining file:///content/shap-e
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-install-ih8ozl_h/clip_7c00c5ab6f904929a823f2871b3e53eb
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-install-ih8ozl_h/clip_7c00c5ab6f904929a823f2871b3e53eb
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.7/73.7 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 5.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_6

In [4]:
import torch

from shap_e.diffusion.sample import sample_latents
from shap_e.diffusion.gaussian_diffusion import diffusion_from_config
from shap_e.models.download import load_model, load_config
from shap_e.util.notebooks import create_pan_cameras, decode_latent_images, gif_widget

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
xm = load_model('transmitter', device=device)
model = load_model('text300M', device=device)
diffusion = diffusion_from_config(load_config('diffusion'))

  0%|          | 0.00/1.78G [00:00<?, ?iB/s]

100%|███████████████████████████████████████| 890M/890M [00:09<00:00, 97.6MiB/s]


  0%|          | 0.00/1.26G [00:00<?, ?iB/s]

In [7]:
batch_size = 8
guidance_scale = 15.0
prompt = "A Transparant cylinderical shape looking circular from back side and empty from front side"

latents = sample_latents(
    batch_size=batch_size,
    model=model,
    diffusion=diffusion,
    guidance_scale=guidance_scale,
    model_kwargs=dict(texts=[prompt] * batch_size),
    progress=True,
    clip_denoised=True,
    use_fp16=True,
    use_karras=True,
    karras_steps=64,
    sigma_min=1e-3,
    sigma_max=160,
    s_churn=0,
)

  0%|          | 0/64 [00:00<?, ?it/s]

In [8]:
render_mode = 'nerf' # you can change this to 'stf'
size = 64 # this is the size of the renders; higher values take longer to render.

cameras = create_pan_cameras(size, device)
for i, latent in enumerate(latents):
    images = decode_latent_images(xm, latent, cameras, rendering_mode=render_mode)
    display(gif_widget(images))

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIcAAN7e393e397d397d3t3d393d3tzd3tzd3d3c3tzc3tzc3dvc3d…

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIcAAOnp6eno6ejo6ejo6Ojn6Ofn6Ofn5+bn5+bm5+bm5uXm5uTl5e…

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIcAAONxwd9wvNptutltudZvt9dtudZqtNVrtdNst9NptdJstdJstN…

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIcAAJiGo5iFpJiFo5eFpJeFo5qEppqEpZiEpJeEo5aFpZaFpJaFo5…

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIcAAPDx8uzu7e3t7uzt7uvt7Onr6+rq6+nq6ujp6efo6ebn5uXm5e…

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIcAAKS6xaS5xaS5xKO5xaO5xKS4xKO4xKK5xaK5xKK4xKG5xaG4xa…

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIcAAOjo6ejo6Ojn6Ofn6Ofn5+bm5+bm5uXm5+Xm5uXl5eTl5uTl5e…

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIcAAOfn5+bn5+bm5+Tl5uHk5eLj5eHi4uDj5ODh4+Dh4d/h49/h4t…

In [ ]:
# Example of saving the latents as meshes.
from shap_e.util.notebooks import decode_latent_mesh

for i, latent in enumerate(latents):
    t = decode_latent_mesh(xm, latent).tri_mesh()
    with open(f'example_mesh_{i}.ply', 'wb') as f:
        t.write_ply(f)
    with open(f'example_mesh_{i}.obj', 'w') as f:
        t.write_obj(f)